In [ ]:
!pip install -q transformers accelerate faiss-cpu sentence-transformers

import torch
import pandas as pd
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from google.colab import drive
import warnings

warnings.filterwarnings("ignore")

mistral_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
mistral_token = "hXXXXXXXXXXXXXXXXXXXXXX"

mistral_tokenizer = AutoTokenizer.from_pretrained(mistral_model_id, token=mistral_token)
mistral_model = AutoModelForCausalLM.from_pretrained(
    mistral_model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    token=mistral_token
)

classifier_model_id = "distilbert-base-uncased-finetuned-sst-2-english"
classifier_tokenizer = AutoTokenizer.from_pretrained(classifier_model_id)
classifier_model = AutoModelForSequenceClassification.from_pretrained(classifier_model_id).to("cuda")

drive.mount('/content/drive')
csv_path = "/content/drive/MyDrive/email_spam.csv"

try:
    df = pd.read_csv(csv_path, encoding='ISO-8859-1')
except FileNotFoundError:
    print(f"ERROR: The file was not found at {csv_path}. Please check the path.")
    df = pd.DataFrame({
        'type': ['ham', 'spam'],
        'text': ['Hi, how are you?', 'Click here to win a free prize!']
    })


assert 'text' in df.columns and 'type' in df.columns, "CSV must contain 'text' and 'type' columns"
documents = df['text'].fillna("").tolist()
labels = df['type'].fillna("").tolist()

print("Embedding documents for retrieval...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = embedder.encode(documents, show_progress_bar=True, convert_to_tensor=True)
corpus_embeddings = corpus_embeddings.cpu().numpy().astype("float32")

index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)
print("FAISS index created successfully.")

def retrieve_docs(query, k=3):
    """Retrieves the most relevant documents from the FAISS index."""
    query_embedding = embedder.encode([query]).astype("float32")
    _, indices = index.search(query_embedding, k)
    # Return concise context for the prompt
    return [f"[{labels[i].upper()}] {documents[i][:350]}..." for i in indices[0]]

def classify_spam_fast(email_text):
    """
    Uses the fast DistilBERT model to get an immediate classification.
    Returns the verdict ('SPAM' or 'NOT SPAM') and confidence.
    """
    inputs = classifier_tokenizer(email_text, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    with torch.no_grad():
        logits = classifier_model(**inputs).logits


    predicted_class_id = logits.argmax().item()
    confidence = logits.softmax(dim=1)[0][predicted_class_id].item()

    verdict = "SPAM" if predicted_class_id == 0 else "NOT SPAM"
    return verdict, confidence


def generate_reasoning(query, verdict, k=3):
    """
    Uses Mistral to generate a justification for the classification provided by the faster model.
    """
    context_docs = retrieve_docs(query, k=k)
    context = "\n\n".join(context_docs)

    prompt = f"""<s>[INST] An email has been classified as **{verdict}**.
Based on the context of similar past emails provided below, give a brief, one-paragraph explanation for this classification.
Focus on the tone, keywords, sense of urgency, and calls to action in the new email.
Do not repeat the provided context or the original email in your response.

### Context from Past Emails:
{context}

### New Email to Analyze:
"{query}"
[/INST] Reasoning:"""

    inputs = mistral_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to("cuda")


    outputs = mistral_model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.15,
        pad_token_id=mistral_tokenizer.eos_token_id
    )

    # Faster and cleaner decoding
    input_length = inputs['input_ids'].shape[1]
    result = mistral_tokenizer.decode(outputs[0, input_length:], skip_special_tokens=True)

    return result.strip()
print("\nMulti-Model RAG Chatbot Ready! I'll classify your email and explain why. Type 'exit' to quit.\n")

while True:
    query = input("You (enter email text): ")
    if query.lower() in ["exit", "quit"]:
        break
    if not query.strip():
        continue
    verdict, confidence = classify_spam_fast(query)

    print(f"\n**Verdict:** {verdict} (Confidence: {confidence:.2%})")
    print("Mistral is generating the reasoning...")

    reasoning = generate_reasoning(query, verdict)
    print(f"\n**Reasoning:**\n{reasoning}\n")

    # Clean up GPU memory
    torch.cuda.empty_cache()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.7 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

FAISS index created successfully.

Multi-Model RAG Chatbot Ready! I'll classify your email and explain why. Type 'exit' to quit.

You (enter email text): Dear Swayam Swarup Barik,  Congratulations! Thank you for applying to the Business Analysis Hands-On-Experience Training Program at Victoria Solutions.   We’re excited to inform you that you’ve been officially selected to join the Business Analysis Hands-On-Experience Training Program at Victoria Solutions.  This exclusive internship and training is designed to bridge the gap between academic learning and real-world industry demands. You’ll gain hands-on experience, build your portfolio, and receive the guidance you need to thrive in today’s job market.  Why Enroll Now? Here’s what awaits you: Job Opportunities: Priority consideration for roles within Victoria Solutions and other client companies. Training Experience Letter – An official document to enhance your professional portfolio. 4 Certificates of Achievement: Including completi